> LSA (영어)

- 특이값 분해(Singular Value Decomposition, SVD)
- 차원축소 방법 중 하나
- m x n 행렬을 m x m 직교행렬, n x n 직교행렬, m x n 직사각 대각행렬 3개의 곱으로 분해하는 작업
- 행렬 연산을 통해 데이터의 차원을 축소하고 중요한 특징들을 추출하는 기법

In [2]:
#20가지 주제의 뉴스 데이터
import pandas as pd
# from sklearn.datasets import fetch_20newsgroups

# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                             remove=('headers', 'footers','quotes'))
# documents = dataset.data

f = open('c:/vscode/data/news/news2.txt')
lines = f.readlines()
f.close()

In [3]:
news_df = pd.DataFrame({'document':lines})

# 알파벳 이외의 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

# 길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x:x.lower())

news_df['clean_doc'][0]

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_9488\3234014594.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


'recent days boss russian private military company wagner seems have gone into social media meltdown flooding telegram channel other accounts with ever more outrageous provocative statements'

In [4]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# 토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x:x.split())

# 불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: 
                                    [item for item in x if item not in stop_words])
print(tokenized_doc[0])

['recent', 'days', 'boss', 'russian', 'private', 'military', 'company', 'wagner', 'seems', 'gone', 'social', 'media', 'meltdown', 'flooding', 'telegram', 'channel', 'accounts', 'ever', 'outrageous', 'provocative', 'statements']


In [5]:
# tf-idf 행렬을 만들기 위해 다시 역토큰화
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
news_df['clean_doc'] = detokenized_doc
news_df['clean_doc'][0]

'recent days boss russian private military company wagner seems gone social media meltdown flooding telegram channel accounts ever outrageous provocative statements'

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

#상위 1000개의 단어만 처리
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape     # TF-IDF 행렬의 크기 확인

(42, 329)

In [11]:
from sklearn.decomposition import TruncatedSVD

# 행렬 특이값 분해, 11314개의 행을 20개로 축소, n_components 토픽수
svd_model = TruncatedSVD(n_components=5)
svd_model.fit(X)
len(svd_model.components_)

5

In [12]:
import numpy as np

#토픽수 x 단어수
np.shape(svd_model.components_)

(5, 329)

In [13]:
svd_model.components_

array([[ 0.01007857,  0.01007857,  0.05542062, ...,  0.01007857,
         0.01007857,  0.01765673],
       [ 0.00737184,  0.00737184, -0.03977332, ...,  0.00737184,
         0.00737184,  0.00095199],
       [ 0.02616046,  0.02616046, -0.04249638, ...,  0.02616046,
         0.02616046,  0.00750172],
       [ 0.00242608,  0.00242608, -0.05317281, ...,  0.00242608,
         0.00242608,  0.01736617],
       [-0.00739423, -0.00739423,  0.01188695, ..., -0.00739423,
        -0.00739423, -0.02618762]])

In [14]:
# 단어 집합, 1000개의 단어
terms = vectorizer.get_feature_names_out()

#20개의 뉴스그룹별로 추출한 토픽 리스트 출력
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1),
                [(feature_names[i], topic[i].round(5)) 
                for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_,terms)
#각 토픽의 핵심 키워드 추출

# LSA: 쉽고 빠르게 구현이 가능하지만 새로운 데이터가 추가되면
# 처음부터 다시 계산을 해야 하는 단점이 있음

Topic 1: [('prigozhin', 0.2978), ('military', 0.29017), ('russia', 0.22763), ('putin', 0.1885), ('russian', 0.18648)]
Topic 2: [('putin', 0.4366), ('expendable', 0.23777), ('politically', 0.23777), ('prigozhin', 0.15094), ('kremlin', 0.13324)]
Topic 3: [('shells', 0.41089), ('save', 0.1733), ('warehouses', 0.16706), ('grandpa', 0.14005), ('receive', 0.14005)]
Topic 4: [('military', 0.23048), ('public', 0.21875), ('leadership', 0.19947), ('criticism', 0.1655), ('continued', 0.1607)]
Topic 5: [('politically', 0.47309), ('expendable', 0.47309), ('star', 0.19444), ('political', 0.19093), ('shells', 0.11903)]


> LSD (한글)

In [18]:
import pandas as pd
f = open("c:/vscode/data/news/news1.txt")
lines = f.readlines()
f.close()

In [19]:
import re
from konlpy.tag import Hannanum

han = Hannanum()
word_list = []
for i in range(len(lines)):
    sentence = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]',' ', lines[i])
    
    #명사만 추출
    a = sentence.strip()
    n = han.nouns(a)
    n2 = [x for x in n if len(x)>1]
    if len(n2)>0:
        word_list.append(n2)

# tf-idf 행렬만들기위해 다시 역토큰화
detokenized_doc = []
for i in range(len(word_list)):
    t = ' '.join(word_list[i])
    detokenized_doc.append(t)
detokenized_doc

['서울 상위 고급 주택 가격 집계 주요 도시 하락폭',
 '현지시간 영국 부동산 정보업체 나이트프랭크 발표 최고급 글로벌 도시 지수 분기 서울 상위 고급 주택 가격 전년 기간 대비 하락 개월 대비 하락폭 개월',
 '상반 서울 상위 고급 주택 가격 지난해 분기 전년 동기 대비 전체 도시 아시 도시들 도쿄',
 '세계 주요 도시 대상 가격 기준 상위 주택 가격 추이 조사',
 '분기 주요 도시 고급 주택 가격 평균 전년 동기 대비 하락 글로벌 금융위 이후 처음 분기 지난해 분기 상승 지난해 분기 상승률',
 '도시 절반 고급 주택 가격 상승률 대부분 자릿수 제외 상승률',
 '전체 가격 평균 가격 하락 도시 낙폭 커서 이중 하락률 하락폭 도시 뉴질랜드 웰링턴 대비 고급 주택 가격 급락했다 토론토 스웨덴 스톡홀름 가격',
 '미국 주요 도시들 고급 주택 가격 하락세 타격 도시 실리콘밸리 샌프란시스코 대비 고급 주택 가격 뉴욕 로스앤젤레스 하락',
 '리암 베일리 나이트프랭크 연구책임자 세계 주요국 통화긴축 정책 일환 기준금리 인상한 세계 고급 주택 가격 하락',
 '두바 마이애미 예외적 고급 주택 상승률 자릿수 기록 두바 적극적 외국 유치 정책 이스라엘 각국 투자자들 유치 집값 급등',
 '나이트프랭크 세계 부동산 시장 하락세 우려 심각 중앙은행 기준금리 동결 전망 한몫',
 '베일리 세계 중앙은행들 금리 동결 글로벌 집값 분기 동안 하락 압력 현재 글로벌 금융위 집값 급락할 평가']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(detokenized_doc)
X.shape

(12, 98)

In [26]:
from sklearn.decomposition import TruncatedSVD

# n_components 토픽수
svd_model = TruncatedSVD(n_components=3)
svd_model.fit(X)
len(svd_model.components_)

3

In [27]:
import numpy as np

#토픽수 x 단어수
np.shape(svd_model.components_)

(3, 98)

In [28]:
# 단어 집합
terms = vectorizer.get_feature_names()
def get_topics(components, feature_names, n=3):
    for idx, topic in enumerate(components):
        print('Topic %d:' % (idx+1),
            [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])
        
get_topics(svd_model.components_, terms)
#각 토픽의 핵심 키워드 추출

Topic 1: [('가격', 0.38663), ('도시', 0.28693), ('주택', 0.2705)]
Topic 2: [('세계', 0.36519), ('기준금리', 0.2504), ('동결', 0.23015)]
Topic 3: [('분기', 0.46197), ('지난해', 0.27615), ('글로벌', 0.25137)]


> LDA (영어)

- 잠재 디리클레 할당
- 주제별 단어수 분포를 바탕으로 주어진 문서에서 발견된 단어수 분포를 분석하여 해당 문서의 주제들을 예측하는 기법
- 교환성 가정 : 단어들의 순서는 상관하지 않고 단어들의 유무만이 중요하다는 가정

In [29]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# 영어 단어의 어근만 추출
stm = PorterStemmer()

# 영어 단어의 불용어 집합
stopwords = set(stopwords.words('english'))

# 첫글자는 알파벳으로 시작하고 그 뒤에 영문자 대소문자, 숫자, -, _, .만 허용
# * 0회 이상 매칭, + 1회 이상 매칭
pattern = re.compile('[a-zA-Z][-_a-zA-Z0-9.]*')

# 문장을 단어 단위로 분리하고 불용어 및 특수문자 제거 후 
# 어근만 추출하여 list로 반환
def tokenize(sentence):
    def stem(w):
        try: return stm.stem(w)
        except: return w
    
    #소문자로 바꾼 후 단어 구분, 불용어 제거, 패턴에 맞는 단어만 선택
    return [stem(w) for w in word_tokenize(sentence.lower())
            if w not in stopwords and pattern.match(w)]

In [39]:
import tomotopy as tp
# LDAModel 생성
# LDA(Latent Dirichlet allocation, 잠재 디리클레 할당)
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 
# 서술하는 확률적 토픽 모델 기법
# 토픽의 개수(k) 5개, 5회 미만 등장한 단어들은 제거
model = tp.LDAModel(k=5, min_cf=5)
# 파일에서 한 줄씩 읽어와서 model에 추가
for i, line in enumerate(open('c:/vscode/data/text/warandpeace.txt')):
    model.add_doc(tokenize(line)) # 공백 기준으로 단어를 나누어 model에 추가

# train(0) : 0회 학습, model의 num_words, num_vocabs 값을 확인하기 위해
# 실제로 학습은 하지 않고 학습 준비만 하는 상태
model.train(0)
print('Total docs:', len(model.docs))
print('Total words:', model.num_words)
print(model.vocabs) #단어 리스트

Total docs: 12172
Total words: 259099
['said', 'one', 'pierr', 'princ', 'look', 'would', 'natásha', 'man', 'andrew', 'could', 'face', 'time', 'go', 'know', 'say', 'ask', 'princess', 'rostóv', 'see', 'thought', 'come', 'eye', 'french', 'hand', 'went', 'like', 'room', 'old', 'men', 'day', 'seem', 'count', 'chapter', 'smile', 'even', 'began', 'moscow', 'armi', 'turn', 'well', 'came', 'offic', 'still', 'mari', 'without', 'gener', 'emperor', 'love', 'felt', 'word', 'nichola', 'head', 'first', 'anoth', 'away', 'order', 'left', 'someth', 'russian', 'life', 'two', 'peopl', 'napoleon', 'take', 'voic', 'command', 'littl', 'ye', 'talk', 'soldier', 'hors', 'feel', 'long', 'whole', 'express', 'kutúzov', 'back', 'way', 'heard', 'think', 'repli', 'countess', 'must', 'moment', 'wish', 'right', 'alway', 'saw', 'noth', 'want', 'made', 'thing', 'tell', 'hous', 'get', 'let', 'though', 'young', 'sónya', 'place', 'move', 'side', 'father', 'suddenli', 'round', 'evid', 'tri', 'understand', 'good', 'denísov', 

In [40]:
#200회 학습
model.train(200)
# 학습된 토픽들을 출력
for i in range(model.k):
    # 0~19번 토픽별 상위 단어 10개 추출
    res = model.get_topic_words(i, top_n=10)
    print(f'Topic #{i}', end='\t')
    print(', '.join(w for w, p in res))

Topic #0	princ, pierr, would, andrew, princess, thought, natásha, love, mari, day
Topic #1	one, armi, man, napoleon, emperor, peopl, russian, men, would, moscow
Topic #2	look, face, eye, pierr, hand, room, head, went, one, natásha
Topic #3	said, chapter, go, ask, know, ye, well, say, come, let
Topic #4	offic, hors, french, soldier, one, fire, command, men, order, left
